In [1]:
import pandas as pd
import json

In [2]:
matches = pd.read_csv("../data/all_years_with_new_columns/matches.csv")
with open("../data/tennis_explorer_scraped/scraped_match_time.json") as f:
    scraped_time_json = json.load(f)

In [3]:
matches_2021 = matches[matches['match_id'].str.split("_").str[1] == "2021"]

In [4]:
scraped_time_json

{'Delray Beach': {'1. round': [{'Player1': 'Mager Gianluca',
    'Player2': 'Harrison Ryan',
    'Time': '20:40',
    'Date': '07.01.2021'},
   {'Player1': 'Paul Tommy',
    'Player2': 'Nam Ji Sung',
    'Time': '18:40',
    'Date': '07.01.2021'},
   {'Player1': 'Monteiro Thiago',
    'Player2': 'Bellucci Thomaz',
    'Time': '17:15',
    'Date': '07.01.2021'},
   {'Player1': 'Norrie Cameron',
    'Player2': 'Aragone JC',
    'Time': '20:45',
    'Date': '08.01.2021'},
   {'Player1': 'Fratangelo Bjorn',
    'Player2': 'King Kevin',
    'Time': '20:15',
    'Date': '08.01.2021'},
   {'Player1': 'Quiroz Roberto',
    'Player2': 'Rubin Noah',
    'Time': '18:55',
    'Date': '08.01.2021'},
   {'Player1': 'Galan Daniel Elahi',
    'Player2': 'Martin Andrej',
    'Time': '18:40',
    'Date': '08.01.2021'},
   {'Player1': 'Harrison Christian',
    'Player2': 'Etcheverry Tomas Martin',
    'Time': '17:05',
    'Date': '08.01.2021'},
   {'Player1': 'Querrey Sam',
    'Player2': 'McDonald Macke

In [5]:
winner_dict = dict(zip(matches['winner_name'], matches['winner_id']))
loser_dict = dict(zip(matches['loser_name'], matches['loser_id']))

# Combine the two dictionaries, with winner_dict taking priority
player_dict = {**loser_dict, **winner_dict}
player_dict

{'Richard Gasquet': 104755,
 'Daniel Elahi Galan': 123755,
 'Christopher Oconnell': 106331,
 'Holger Rune': 208029,
 'Felix Auger Aliassime': 200000,
 'Jordan Thompson': 111442,
 'Mikael Ymer': 144707,
 'Soon Woo Kwon': 126952,
 'Maxime Cressy': 202195,
 'Chun Hsin Tseng': 202358,
 'Bernabe Zapata Miralles': 126523,
 'Manas Dhamne': 211768,
 'Jaume Munar': 144719,
 'Alex Molcan': 144684,
 'Nikoloz Basilashvili': 105932,
 'Sumit Nagal': 111576,
 'Lorenzo Sonego': 132283,
 'Pedro Cachin': 106398,
 'Constant Lestienne': 106005,
 'Kyle Edmund': 106378,
 'Rinky Hijikata': 208014,
 'Andrey Rublev': 126094,
 'Andy Murray': 104918,
 'Radu Albot': 105430,
 'Elias Ymer': 111200,
 'Ramkumar Ramanathan': 106368,
 'Pablo Andujar': 104665,
 'Sasi Kumar Mukund': 124154,
 'Jack Draper': 207733,
 'Mackenzie Mcdonald': 111456,
 'Miomir Kecmanovic': 200175,
 'Marcos Giron': 106218,
 'Laslo Djere': 111513,
 'Tim Van Rijthoven': 126646,
 'Sebastian Baez': 202104,
 'Marco Cecchinato': 106065,
 'Flavio Cobol

In [6]:
for player in player_dict:
    if "-" in player:
        print(player)

Jo-Wilfried Tsonga


In [7]:
def normalize_string(s):
    return " ".join(sorted(s.lower().split()))

# Normalize the dictionary keys
normalized_player_dict = {normalize_string(key): value for key, value in player_dict.items()}

def get_player_id(player_name):
    normalized_player_name = normalize_string(player_name)
    return normalized_player_dict.get(normalized_player_name)

In [8]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            if get_player_id(match['Player1']) is None:
                print(match['Player1'])
            if get_player_id(match['Player2']) is None:
                print(match['Player2'])

Herbert Pierre-Hugues
Struff Jan-Lennard
Struff Jan-Lennard
Struff Jan-Lennard
Stebe Cedrik-Marcel
Carreno-Busta Pablo
Ramos-Vinolas Albert
Lu Yen-hsun
Herbert Pierre-Hugues
Bautista-Agut Roberto
O'Connell Christopher
Struff Jan-Lennard
Carreno-Busta Pablo
O'Connell Christopher
Carreno-Busta Pablo
Ramos-Vinolas Albert
Barrios Vera Marcelo Tomas
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Kwiatkowski Thai-Son
Struff Jan-Lennard
Bautista-Agut Roberto
Bautista-Agut Roberto
Bautista-Agut Roberto
Bautista-Agut Roberto
Bautista-Agut Roberto
Struff Jan-Lennard
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Bautista-Agut Roberto
O'Connell Christopher
Bautista-Agut Roberto
Bautista-Agut Roberto
Bautista-Agut Roberto
Bautista-Agut Roberto
Herbert Pierre-Hugues
Herbert Pierre-Hugues
Herbert Pierre-Hugues
Herbert Pierre-Hugues
Herbert Pierre-Hugues
Ramos-Vinolas Albert
O'Connell Christopher
Struff Jan-Lennard
Carreno-Bust

## fixing player names in json

In [9]:
def fix_player_names_json(name: str):
    if name == "Ramos-Vinolas Albert":
        return "Ramos Albert"
    if name == "O'Connell Christopher":
        return "Oconnell Christopher"
    if name == "Barrios Vera Marcelo Tomas":
        return "Barrios Vera Tomas"
    if name == "Wolf Jeffrey John":
        return "J J Wolf"
    if name != "Tsonga Jo-Wilfried" and "-" in name:
        return name.replace("-", " ")
    return name

In [10]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match['Player1'] = fix_player_names_json(match['Player1'])
            match['Player2'] = fix_player_names_json(match['Player2'])
            if get_player_id(match['Player1']) is None:
                print(match['Player1'])
            if get_player_id(match['Player2']) is None:
                print(match['Player2'])

### Możemy w takim razie dodać player id z matches df do scraped_time_json

In [11]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match['Player1Id'] = get_player_id(match['Player1'])
            match['Player2Id'] = get_player_id(match['Player2'])

### Będziemy teraz chcieli uzyskać takie same wartości tournament w jsonie jak tournament_location w matches_df

In [12]:
matches_df_locations = set(matches_2021['tournament_location'].unique())
json_locations = []
for tournament in scraped_time_json:
    json_locations.append(tournament)
    
only_in_one_df = matches_df_locations ^ set(json_locations)
if only_in_one_df:
    print(f"Locations only in one df: {only_in_one_df}")
else:
    print("Both dfs  have the same tournament locations.")

Locations only in one df: {'Astana', 'Winston Salem', 'Nur-Sultan', "Queen's Club", 'KitzbĂĽhel', 'Saint Petersburg', 'Kitzbuhel', 'Winston-Salem', 'St. Petersburg', 'Queens Club'}


In [13]:
def fix_tournament_name(name):
    if name == "Queen's Club":
        return "Queens Club"
    if name == "Kitzbühel":
        return "Kitzbuhel"
    if name == "KitzbĂĽhel": # For some reason ü can be turned into this
        return "Kitzbuhel"
    if name == "Winston Salem":
        return "Winston-Salem"
    if name == "Astana":
        return "Nur-Sultan"
    if name == "Saint Petersburg":
        return "St. Petersburg"
    return name

In [14]:
matches_df_locations = set(matches_2021['tournament_location'].unique())
json_locations = []
for tournament in scraped_time_json:
    tournament = fix_tournament_name(tournament)
    json_locations.append(tournament)
    
only_in_one_df = matches_df_locations ^ set(json_locations)
if only_in_one_df:
    print(f"Locations only in one df: {only_in_one_df}")
else:
    print("Both dfs  have the same tournament locations.")

Both dfs  have the same tournament locations.


### Możemy zatem zrobić match_id takie jak w matches df

In [15]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match['match_id'] = fix_tournament_name(tournament) + '_' + "2021" + '_' + str(match['Player1Id']) + '_' + str(match['Player2Id'])

### Sprawdźmy czy match_id jest unikatowy

In [16]:
match_ids = []
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match_ids.append(match["match_id"])
print(len(match_ids) == len(set(match_ids)))

True


### Sprawdźmy teraz czy jest są jakieś match_id tylko w jednej z ramek

In [17]:
for match_id in match_ids:
    if match_id not in matches_2021['match_id'].values:
        print(match_id)

In [18]:
for match_id in matches_2021['match_id'].values:
    if match_id not in match_ids:
        print(match_id)

### Nic sie nie printuje więc jest git

## 2022

In [19]:
matches_2022 = matches[matches['match_id'].str.split("_").str[1] == "2022"]

In [20]:
with open("../data/tennis_explorer_scraped/scraped_match_time_2022.json") as f:
    scraped_time_json = json.load(f)

In [21]:
scraped_time_json

{'Adelaide': {'1. round': [{'Player1': 'Johnson Steve',
    'Player2': 'Vukic Aleksandar',
    'Time': '12:20',
    'Date': '03.01.2022'},
   {'Player1': 'Djere Laslo',
    'Player2': 'Carballes Baena Roberto',
    'Time': '06:20',
    'Date': '03.01.2022'},
   {'Player1': 'Monteiro Thiago',
    'Player2': 'Altmaier Daniel',
    'Time': '03:05',
    'Date': '03.01.2022'},
   {'Player1': 'Kwon Soon Woo',
    'Player2': 'Nishioka Yoshihito',
    'Time': '01:40',
    'Date': '03.01.2022'},
   {'Player1': 'Kokkinakis Thanasi',
    'Player2': 'Millman John',
    'Time': '09:40',
    'Date': '04.01.2022'},
   {'Player1': 'Mager Gianluca',
    'Player2': 'Cerundolo Francisco',
    'Time': '09:05',
    'Date': '04.01.2022'},
   {'Player1': 'Paul Tommy',
    'Player2': 'Vesely Jiri',
    'Time': '06:35',
    'Date': '04.01.2022'},
   {'Player1': 'Cerundolo Juan Manuel',
    'Player2': 'Bolt Alex',
    'Time': '06:15',
    'Date': '04.01.2022'},
   {'Player1': 'Ymer Mikael',
    'Player2': 'Bonz

In [22]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            if get_player_id(match['Player1']) is None:
                print(match['Player1'])
            if get_player_id(match['Player2']) is None:
                print(match['Player2'])

O'Connell Christopher
Struff Jan-Lennard
Carreno-Busta Pablo
Ramos-Vinolas Albert
Barrios Vera Marcelo Tomas
Struff Jan-Lennard
Bautista-Agut Roberto
O'Connell Christopher
Carreno-Busta Pablo
Bautista-Agut Roberto
O'Connell Christopher
Carreno-Busta Pablo
O'Connell Christopher
Bautista-Agut Roberto
Carreno-Busta Pablo
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Herbert Pierre-Hugues
Bautista-Agut Roberto
Herbert Pierre-Hugues
Bautista-Agut Roberto
Stebe Cedrik-Marcel
Stebe Cedrik-Marcel
Ramos-Vinolas Albert
Del Potro Juan-Martin
Ramos-Vinolas Albert
Meligeni Rodrigues Alves Felipe
Ramos-Vinolas Albert
Carreno-Busta Pablo
O'Connell Christopher
Bautista-Agut Roberto
Bautista-Agut Roberto
Bautista-Agut Roberto
Bautista-Agut Roberto
Herbert Pierre-Hugues
Herbert Pierre-Hugues
O'Connell Christopher
Bautista-Agut Roberto
Struff Jan-Lennard
Bautista-Agut Roberto
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Ramos-Vinolas Albert
Carreno-

In [23]:
def fix_player_names_json(name: str):
    if name == "Ramos-Vinolas Albert":
        return "Ramos Albert"
    if name == "O'Connell Christopher":
        return "Oconnell Christopher"
    if name == "Barrios Vera Marcelo Tomas":
        return "Barrios Vera Tomas"
    if name == "Wolf Jeffrey John":
        return "J J Wolf"
    if name == "Meligeni Rodrigues Alves Felipe":
        return "Meligeni Alves Felipe"
    if name == "Rodriguez Taverna Santiago":
        return "Rodriguez Taverna Santiago Fa"
    if name != "Tsonga Jo-Wilfried" and "-" in name:
        return name.replace("-", " ")
    return name

In [24]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match['Player1'] = fix_player_names_json(match['Player1'])
            match['Player2'] = fix_player_names_json(match['Player2'])
            if get_player_id(match['Player1']) is None:
                print(match['Player1'])
            if get_player_id(match['Player2']) is None:
                print(match['Player2'])

In [25]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match['Player1Id'] = get_player_id(match['Player1'])
            match['Player2Id'] = get_player_id(match['Player2'])

In [26]:
locations = matches_2022['tournament_location'].unique()
for tournament in scraped_time_json:
    if tournament not in locations:
        print(tournament)

Adelaide
Australian Open
French Open
Hertogenbosch
Queen's Club
Wimbledon
KitzbĂĽhel
Winston Salem
US Open
Tokyo (Japan Open)
Astana
Napoli


In [27]:
def fix_tournament_name(name):
    if name == "Queen's Club":
        return "Queens Club"
    if name == "Kitzbühel":
        return "Kitzbuhel"
    if name == "KitzbĂĽhel": # For some reason ü can be turned into this
        return "Kitzbuhel"
    if name == "Winston Salem":
        return "Winston-Salem"
    if name == "Astana":
        return "Nur-Sultan"
    if name == "Saint Petersburg":
        return "St. Petersburg"
    if name == "US Open":
        return "New York"
    if name == "Australian Open":
        return "Melbourne"
    if name == "Wimbledon":
        return "London"
    if name == "French Open":
        return "Paris"
    if name == "Paris":
        return "Paris 2"
    if name == "Tokyo (Japan Open)":
        return "Tokyo"
    if name == "Napoli":
        return "Naples"
    if name == "Hertogenbosch":
        return "'s-Hertogenbosch"
    if name == "Adelaide":
        return "Adelaide 1"
    return name

In [28]:
matches_df_locations = set(matches_2022['tournament_location'].unique())
json_locations = []
for tournament in scraped_time_json:
    tournament = fix_tournament_name(tournament)
    json_locations.append(tournament)
    
only_in_one_df = matches_df_locations ^ set(json_locations)
if only_in_one_df:
    print(f"Locations only in one df: {only_in_one_df}")
else:
    print("Both dfs  have the same tournament locations.")

Both dfs  have the same tournament locations.


In [29]:
locations = matches_2022['tournament_location'].unique()
for tournament in scraped_time_json:
    tournament = fix_tournament_name(tournament)
    if tournament not in locations:
        print(tournament)

In [30]:
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match['match_id'] = fix_tournament_name(tournament) + '_' + "2022" + '_' + str(match['Player1Id']) + '_' + str(match['Player2Id'])

In [31]:
match_ids = []
for tournament in scraped_time_json:
    for round in scraped_time_json[tournament]:
        for match in scraped_time_json[tournament][round]:
            match_ids.append(match["match_id"])
print(len(match_ids) == len(set(match_ids)))

True


In [32]:
for match_id in match_ids:
    if match_id not in matches_2022['match_id'].values:
        print(match_id)

In [33]:
for match_id in matches_2022['match_id'].values:
    if match_id not in match_ids:
        print(match_id)